# Drowsiness Detection

In [ ]:
from scipy.spatial import distance as dist
from imutils import face_utils
import imutils
import dlib
import cv2
import winsound #for generating sound

frequency = 2500 
duration = 1000

In [ ]:
def eyeAspectRatio(eye):
    #vertical coordinates of eye
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    #horizontal coordinates of eye
    C = dist.euclidean(eye[0], eye[3])
    ear = (A+B) / (2.0 * C)
    return ear

In [ ]:
count = 0 
eye_aspect_ratio_thres = 0.3 #diatance between vertical eye coordinate threshold
eyeFrames = 48 #consecutive frames for eye closure

shapePredictor = "shape_predictor_81_face_landmarks.dat"

cam = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shapePredictor)

#get the coord of left and right eye

(lstart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

while True:
    _,frame = cam.read()
    frame = imutils.resize(frame, width = 450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    reacts = detector(gray , 0)
    
    for rect in reacts:
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        
        #combining both the eyes and hence calculate the combined aspect ratio
        leftEye = shape[lstart:lEnd]
        rightEye = shape[rstart:rEnd]
        leftEAR = eyeAspectRatio(leftEye)
        rightEAR = eyeAspectRatio(rightEye)
        
        ear = (leftEAR + rightEAR) / 2.0
        
        #collecting dots of eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        #combining all the points of eye(the dots from above)
        cv2.drawContours(frame, [leftEyeHull], -1 ,(0, 0, 255), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 0, 255), 1)
        
        if ear < eye_aspect_ratio_thres:
            print("b")
            count += 1
            
            if count >= eyeFrames:
                print("A")
                cv2.putText(frame, "DROWSINESS DETECTED", (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
                
                winsound.Beep(frequency, duration)
            
        else:
            count = 0
                
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
        
cam.release()
cv2.destroyAllWindows()